In [ ]:
import numpy as np
import pandas as pd
import fastf1
import statsmodels.api as sm
from src.utils import get_acc_df
from src.plotset import setup_plot, save_fig

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,15,'R')
session.load()

In [ ]:
drivers = session.drivers
drivers = [session.get_driver(drv).Abbreviation for drv in drivers]

In [ ]:
drivers[:5]

In [ ]:
temp = session.laps.pick_drivers('PIA').pick_quicklaps().copy()
temp['Stint_Compound'] = temp.apply(lambda x: str(int(x['Stint'])) + '_'+ x['Compound'],axis=1)

In [ ]:
temp.Stint_Compound

In [ ]:
comp_palette = {'1_SOFT': '#da291c',
                '2_SOFT': '#da291c',
                '3_SOFT': '#da291c',
                '1_MEDIUM': '#ffd12e',
                '2_MEDIUM': '#ffd12e',
                '3_MEDIUM': '#ffd12e',
                '1_HARD': '#f0f0ec',
                '2_HARD': '#f0f0ec',
                '3_HARD': '#f0f0ec'
}

In [ ]:
setup_plot(xyticksize=24,axeslabel=28, figtitle=40)

fig, ax = plt.subplots(figsize=(15,8))

for drv in drivers[:5]:

    df = session.laps.pick_drivers(drv).pick_quicklaps(1.05).copy()
    df['LapTime'] = df['LapTime'].dt.total_seconds()
    df['Stint_Compound'] = df.apply(lambda x: str(int(x['Stint'])) + '_'+ x['Compound'],axis=1)

    color = plotting.get_driver_color(session=session,identifier=drv)

    sns.violinplot(df,x='Driver',y='LapTime',hue='Stint_Compound',palette=[color]*df.Stint.nunique(),
                inner=None,linecolor='w',legend=False)
    sns.stripplot(df,x='Driver',y='LapTime',hue='Stint_Compound',palette=comp_palette
                ,jitter=0.08,linewidth=1,size=15,edgecolor='k',dodge=True,legend=False)


ax.set_ylim(71,78)
ax.set_ylabel("Lap Time (sec)")
ax.set_xlabel("")
ax.set_xticks([])
ax.set_title('Race Pace (lower laptime is better)',pad=30)

ax.grid(visible=False)

In [ ]:
save_fig(fig=fig, name='race_pace',loc='Reel15')

In [ ]:
acc_df = get_acc_df(session=session)[['0-100']]

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

temp = acc_df.iloc[:10].copy()

sns.barplot(data=temp,x=temp.index,y='0-100',hue=temp.index,palette=plotting.get_driver_color_mapping(session=session))

ax.set_title('Time to reach 0-100 km/h (+ reaction time)',pad=20)
ax.set_xlabel('')
ax.set_ylabel('Time (seconds)')
ax.set_ylim(3.15,3.8)